In [8]:
# 查看GPU配置
# Check GPU configuration
!nvidia-smi

Fri May 12 20:44:45 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA RTX A5000    On   | 00000000:06:00.0 Off |                  Off |
| 30%   27C    P8    15W / 230W |      0MiB / 24564MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#STEP 0  
##本笔记本必须在GPU模式下运行，可以运行上方的nvidia-smi查看GPU是否装载。若没有，点击左上角>修改>笔记本设置，把硬件加速改成GPU。  
##！！！开始之前请仔细阅读[数据集上传格式要求](https://github.com/Plachtaa/VITS-fast-fine-tuning/blob/main/DATA.MD)避免后续因为读取不到音频文件而报错。！！！
##Note: GPU is necessary for this notebook. Check whether your GPU is loaded before you start.Please read [Requirements for uploaded data format](https://github.com/Plachtaa/VITS-fast-fine-tuning/blob/main/DATA_EN.MD) carefully to avoid any possible error caused by voice file not found.


In [13]:
#@title STEP 1 复制代码库并安装运行环境
#@markdown #STEP 1 (3 min)
#@markdown ##复制代码库并安装运行环境
#@markdown ##Clone repository & Build environment
#! git clone https://github.com/Plachtaa/VITS-fast-fine-tuning.git
!pip install imageio==2.4.1 -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install --upgrade youtube-dl  -i https://pypi.tuna.tsinghua.edu.cn/simple
!pip install moviepy  -i https://pypi.tuna.tsinghua.edu.cn/simple
%cd VITS-fast-fine-tuning
!pip install -r requirements.txt -i https://pypi.tuna.tsinghua.edu.cn/simple
# build monotonic align
%cd monotonic_align/
!mkdir monotonic_align
!python setup.py build_ext --inplace
%cd ..
!mkdir pretrained_models
# download data for fine-tuning
#!wget https://huggingface.co/datasets/Plachta/sampled_audio4ft/resolve/main/sampled_audio4ft_v2.zip
#!unzip sampled_audio4ft_v2.zip
# create necessary directories
!mkdir video_data
!mkdir raw_audio
!mkdir denoised_audio
!mkdir custom_character_voice
!mkdir segmented_character_voice

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached imageio-2.4.1-py3-none-any.whl
  Attempting uninstall: imageio
    Found existing installation: imageio 2.28.1
    Uninstalling imageio-2.28.1:
      Successfully uninstalled imageio-2.28.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
moviepy 1.0.3 requires imageio<3.0,>=2.5; python_version >= "3.4", but you have imageio 2.4.1 which is incompatible.
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Using cached https://pypi.tuna.tsinghua.edu.cn/packages/e9/67/98ed2279fcd66cc50112abb8f5a3de4526307dc43a16d220d7b48afd113a/imageio-2.28.1-py3-none-any.whl (3.4 MB)
  Attempting uninstall: imageio
    Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.

In [2]:
#@title STEP 1.5 选择预训练模型
#@markdown ###STEP 1.5 选择预训练模型  
#@markdown ###Choose pretrained model to start  
#@markdown CJE为中日英三语模型，CJ为中日双语模型  

#@markdown CJE for Chinese, Japanese & English model，CJ for Chinese & Japanese model
PRETRAINED_MODEL = "CJ" #@param ["CJE","CJ"]
if PRETRAINED_MODEL == "CJ":
  !wget https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/D_0-p.pth -O ./pretrained_models/D_0.pth
  !wget https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/G_0-p.pth -O ./pretrained_models/G_0.pth
  !wget https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/config.json -O ./configs/finetune_speaker.json
elif PRETRAINED_MODEL == "CJE":
  !wget https://huggingface.co/spaces/Plachta/VITS-Umamusume-voice-synthesizer/resolve/main/pretrained_models/D_trilingual.pth -O ./pretrained_models/D_0.pth
  !wget https://huggingface.co/spaces/Plachta/VITS-Umamusume-voice-synthesizer/resolve/main/pretrained_models/G_trilingual.pth -O ./pretrained_models/G_0.pth
  !wget https://huggingface.co/spaces/Plachta/VITS-Umamusume-voice-synthesizer/resolve/main/configs/uma_trilingual.json -O ./configs/finetune_speaker.json

--2023-05-10 22:57:35--  https://huggingface.co/spaces/sayashi/vits-uma-genshin-honkai/resolve/main/model/D_0-p.pth
Resolving huggingface.co (huggingface.co)... 143.204.86.80, 143.204.86.120, 143.204.86.95, ...
Connecting to huggingface.co (huggingface.co)|143.204.86.80|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: /spaces/zomehwh/vits-uma-genshin-honkai/resolve/main/model/D_0-p.pth [following]
--2023-05-10 22:57:36--  https://huggingface.co/spaces/zomehwh/vits-uma-genshin-honkai/resolve/main/model/D_0-p.pth
Reusing existing connection to huggingface.co:443.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/73/57/7357194223863ef0c28d9dd59386c0c4933f0105f0b51e2d107958e679c17cd6/afb0deef1470921019c7edff2e648eee4c35eeae6d4458d1b86fe2ad6c86a4e0?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27D_0-p.pth%3B+filename%3D%22D_0-p.pth%22%3B&Expires=1683987435&Policy=eyJTdGF0ZW1lbnQiOlt7I

In [ ]:
#@title （可选）加载Google云端硬盘 / Mount Google drive
#@title (optional) 

#@markdown 加载Google云端硬盘（更快地上传数据集文件）  

#@markdown Mount Google drive for faster data upload
from google.colab import drive
drive.mount('/content/drive')

## STEP 2 上传您的角色音频数据  
## Upload your character voices
见[数据集上传选项](https://github.com/Plachtaa/VITS-fast-fine-tuning/blob/main/DATA.MD)  
See [data upload options](https://github.com/Plachtaa/VITS-fast-fine-tuning/blob/main/DATA_EN.MD)


###STEP 2.1 上传短音频
### Short audio upload

In [ ]:
#@markdown 上传选项1：运行该代码块会出现一个文件上传的入口  

#@markdown Upload option 1: Running this code block will prompt you to upload a file.
%run voice_upload.py --type zip
!unzip ./custom_character_voice/custom_character_voice.zip -d ./custom_character_voice/

In [ ]:
#@markdown 上传选项2：若您装载了Google云端硬盘，可以直接从Google云端硬盘加载文件。将`.zip`文件上传云端硬盘后，在下面填写文件路径：  

#@markdown Upload option 2: If you have mounted Google drive, you can load your files from Google drive directly. After uploading your `.zip` file to Google drive, fill in the path to your file below:
ZIP_PATH = "../drive/MyDrive/samples.zip"  #@param {type:"string"}
!cp {ZIP_PATH} ./custom_character_voice/custom_character_voice.zip
!unzip ./custom_character_voice/custom_character_voice.zip -d ./custom_character_voice/

### STEP 2.2 上传长音频 （单个不应长于20分钟）
### Long audio upload

In [5]:
#@markdown 上传选项1：运行该代码块会出现一个文件上传的入口  

#@markdown Upload option 1: Running this code block will prompt you to upload a file.
%run voice_upload.py --type audio

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#@markdown 上传选项2：若您装载了Google云端硬盘，可以直接从Google云端硬盘加载文件。将所有长音频文件上传至云端硬盘的同一个文件夹下，在下面填写文件夹路径：  

#@markdown Upload option 2: If you have mounted Google drive, you can load your files from Google drive directly. Put all the long audios under one folder, and fill in the path to your folder below:
AUDIO_FOLDER_PATH = "../drive/MyDrive/long_audios/"  #@param {type:"string"}
!cp {AUDIO_FOLDER_PATH}/* ./raw_audio/

### STEP 2.3 上传视频（单个不应长于20分钟）
### Video upload

In [ ]:
#@markdown 上传选项1：运行该代码块会出现一个文件上传的入口  

#@markdown Upload option 1: Running this code block will prompt you to upload a file.
%run voice_upload.py --type video

In [ ]:
#@markdown 上传选项2：若您装载了Google云端硬盘，可以直接从Google云端硬盘加载文件。将所有视频文件上传至云端硬盘的同一个文件夹下，在下面填写文件夹路径:  

#@markdown Upload option 2: If you have mounted Google drive, you can load your files from Google drive directly. Put all the videos under one folder, and fill in the path to your folder below:
VIDEO_FOLDER_PATH = "../drive/MyDrive/videos/"  #@param {type:"string"}
!cp {VIDEO_FOLDER_PATH}/* ./video_data/

### STEP 2.4 上传视频链接（单个不应长于20分钟）
### Video link upload

In [ ]:
#@markdown 运行该代码块会出现一个文件上传的入口，上传单个`.txt`文件。若格式正确的话，视频会自动下载并将下载后的文件名打印在下方。  

#@markdown Running this code block will prompt you to upload a file. 
#@markdown Please upload a single `.txt` file. If you have put the links in the correct format, 
#@markdown the videos will be automatically downloaded and displayed below.
%run download_video.py
!ls ./video_data/

## STEP 3 自动处理所有上传的数据

In [7]:
%cd VITS-fast-fine-tuning

[Errno 2] No such file or directory: 'VITS-fast-fine-tuning'
/root/VITS-fast-fine-tuning


In [4]:
PRETRAINED_MODEL = "CJ"

In [5]:
#@markdown 运行该单元格会对所有上传的数据进行自动去背景音&标注。 
#@markdown 由于需要调用Whisper和Demucs，运行时间可能较长。  
#@markdown Running this codeblock will perform automatic vocal seperation & annotation. 
#@markdown Since this step uses Whisper & Demucs, it may take a while to complete.
# 将所有视频（无论是上传的还是下载的，且必须是.mp4格式）抽取音频
%run video2audio.py
# 将所有音频（无论是上传的还是从视频抽取的，必须是.wav格式）去噪
!python denoise_audio.py


Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /root/VITS-fast-fine-tuning/separated/htdemucs
Separating track raw_audio/Sxy_00001.wav
100%|████████████████████████████████████████████████████████████████████████| 643.5/643.5 [00:15<00:00, 42.44seconds/s]
Important: the default model was recently changed to `htdemucs` the latest Hybrid Transformer Demucs model. In some cases, this model can actually perform worse than previous models. To get back the old default model use `-n mdx_extra_q`.
Selected model is a bag of 1 models. You will see that many progress bars per track.
Separated tracks will be stored in /root/VITS-fast-fine-tuning/separated/htdemucs
Separating track raw_audio

In [6]:
!rm -rf segmented_character_voice/*
# 分割并标注长音频
!python long_audio_transcribe.py --languages "{PRETRAINED_MODEL}" --whisper_size large
# 标注短音频
!python short_audio_transcribe.py --languages "{PRETRAINED_MODEL}" --whisper_size large

CJ
['Sxy_00001.wav', 'Sxy_00002.wav', 'Sxy_00003.wav']
transcribing ./denoised_audio/Sxy_00001.wav...

{'task': 'transcribe', 'beam_size': 5, 'best_of': 5}
kkkk
Transcribed segment: ./segmented_character_voice/Sxy/Sxy_00001_0.wav|Sxy|[ZH]我们所要介绍的是翔子,不是骆驼,因为骆驼只是个外号,那么我们就先说翔子,随手把骆驼与翔子那点关系说过去也就算了。[ZH]

Transcribed segment: ./segmented_character_voice/Sxy/Sxy_00001_1.wav|Sxy|[ZH]北平的洋车夫有许多派,年轻力壮,腿脚灵力的,讲究漂亮的车,拉着整天爱什么时候出车与收车都有自由,拉出车来,在固定的车口或宅门一放,专等坐车,坐快车的主,弄好了,也许一下子弄个一两块,[ZH]

Transcribed segment: ./segmented_character_voice/Sxy/Sxy_00001_2.wav|Sxy|[ZH]碰巧了也许白号一天连车份也没落着,但也不在乎,这一派哥们的大希望大概有两个,或是拉包车,或是自己买上辆车,有了自己的车,再去拉包月或者散座就没多大关系了,反正是自己的车。[ZH]

Transcribed segment: ./segmented_character_voice/Sxy/Sxy_00001_3.wav|Sxy|[ZH]以这一派最处稍大的,或因身体关系而跑得稍微差点点的,或因家庭关系而不敢白号一天的,大概就是多数拉车,拉八成星的车,人与车都有相当漂亮的,所以在要价的时候也还能保持住相当的尊严。[ZH]

Transcribed segment: ./segmented_character_voice/Sxy/Sxy_00001_4.wav|Sxy|[ZH]这派的车夫,也许拉整天,也许拉半天,在后者的情景下,因为还有相当的精气神,所以无论冬天、夏天总是拉晚儿。[ZH]

Transcribed segment: ./segmented_character_voice/Sxy

In [27]:
#!whisper "/root/VITS-fast-fine-tuning/raw_audio/Sxy_00001.wav" --model tiny --model_dir "/root/VITS-fast-fine-tuning/pretrained_models/w"

Detecting language using up to the first 30 seconds. Use `--language` to specify the language
Detected language: Chinese
[00:00.000 --> 00:03.520] 在擦著漢的時節常常握一看
[00:03.520 --> 00:09.040] 不過他們比較另一些40歲上下的車幅
[00:09.040 --> 00:11.520] 他們事務還沒有補到家
[00:11.520 --> 00:14.280] 這一些以前
[00:14.280 --> 00:18.000] 絕沒有想到自己能與陽車發生關係
[00:18.000 --> 00:22.120] 而到生和死的界限已經分不清
[00:22.120 --> 00:24.840] 才超起車把來的
[00:24.840 --> 00:29.200] 被廁拳、尋減或教育
[00:29.200 --> 00:31.680] 把本錢吃光的小飯
[00:31.680 --> 00:33.480] 或者失業的工人
[00:33.480 --> 00:35.160] 到了脈無可脈
[00:35.160 --> 00:36.920] 當無可
[00:36.920 --> 00:39.160] 當的時候要著牙
[00:39.160 --> 00:40.120] 寒之類
[00:40.120 --> 00:42.600] 走上了這條死亡之路
[00:42.600 --> 00:43.560] 這些人
[00:43.560 --> 00:46.680] 生命最先重的時候已經賣掉
[00:46.680 --> 00:49.720] 現在再把烏烏頭變成
[00:49.720 --> 00:51.640] 換起變成血汗
[00:51.640 --> 00:53.000] 低在馬路上
[00:53.000 --> 00:54.040] 沒有力氣
[00:54.040 --> 00:55.240] 沒有經驗
[00:55.240 --> 00:56.440] 沒有朋友
[00:56.440 --> 01:00.280] 就是在同時當中也得不大好切
[01:00.280 --> 01:01.960] 他們拉最脫的車
[01:01

#！！！训练质量相关：实验发现目前使用CJ模型+勾选ADD_AUXILIARY，对于中/日均能训练出最好的效果，第一次训练建议默认使用该组合！！！

In [7]:
#@markdown ##STEP 3.5 


#@markdown 运行该单元格会生成划分好训练/测试集的最终标注，以及配置文件  

#@markdown Running this block will generate final annotations for training & validation, as well as config file.

#@markdown 选择是否加入辅助训练数据：/ Choose whether to add auxiliary data: 
ADD_AUXILIARY = True #@param {type:"boolean"}
#@markdown 辅助训练数据是从预训练的大数据集抽样得到的，作用在于防止模型在标注不准确的数据上形成错误映射。

#@markdown Auxiliary data is to prevent overfitting when the audio samples are small or with low quality. 

#@markdown 以下情况请勾选：  

#@markdown 总样本少于100条/样本包含角色只有1人/样本质量一般或较差/样本来自爬取的视频  

#@markdown 以下情况可以不勾选：  

#@markdown 总样本量很大/样本质量很高/希望加速训练/只有二次元角色

#@markdown 样本仅包含单说话人会导致报错，请勾选ADD_AUXILIARY或加入几个dummy speaker绕过去  

#@markdown Feeding voice samples containing single speaker will result in error. Please select ADD_AUXILIARY or add several dummy speakers to bypass.

# assert(not (ADD_AUXILIARY and PRETRAINED_MODEL != "CJE")), "add auxiliary data is available only available for CJE model!"
if ADD_AUXILIARY:
  %run preprocess_v2.py --add_auxiliary_data True --languages "{PRETRAINED_MODEL}"
else:
  %run preprocess_v2.py --languages "{PRETRAINED_MODEL}"

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.499 seconds.
Prefix dict has been built successfully.


finished


## STEP 4 开始训练

In [ ]:
#@markdown #STEP 4 (>=20 min)
#@markdown 开始微调模型，在40轮训练后会自动结束。
#@markdown 训练时长取决于你录入/上传的音频总数。

#@markdown 根据声线和样本质量的不同，所需的训练epochs数也不同。但是一般建议设置为40 epochs，  

#@markdown 你也可以在Tensorboard中预览合成效果，若效果满意可提前停止。  

#@markdown Model fine-tuning ends after 40 epochs. 
#@markdown Total time cost depends on the number of voices you recorded/uploaded.   

#@markdown Best epoch number varies depending on different uploaded voices / sample quality. Normally, 40 epochs is suggested.

#@markdown You can also preview synthezied audio in Tensorboard, it's OK to shut down training manually if you find the quality is satisfying.
%reload_ext tensorboard
%tensorboard --logdir "./OUTPUT_MODEL"
Maximum_epochs = "200" #@param [20, 40, 60, 80, 100, 150]
!python finetune_speaker_v2.py -m "./OUTPUT_MODEL" --max_epochs "{Maximum_epochs}" --drop_speaker_embed True

In [2]:
%cd /root/VITS-fast-fine-tuning

/root/VITS-fast-fine-tuning


In [3]:
#@markdown ### 微调完成后，在这里尝试效果。
#@markdown ### Try out TTS & VC quality here after fine-tuning is finished.
!cp ./configs/modified_finetune_speaker.json ./finetune_speaker.json
%run VC_inference.py --model_dir ./OUTPUT_MODEL/G_latest.pth

DEBUG:matplotlib:(private) matplotlib data path: /root/miniconda3/lib/python3.8/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:matplotlib data path: /root/miniconda3/lib/python3.8/site-packages/matplotlib/mpl-data
DEBUG:matplotlib:CONFIGDIR=/root/.config/matplotlib
DEBUG:matplotlib:matplotlib version 3.3.1
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is linux
DEBUG:matplotlib:loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_warnings', '_io', 'marshal', 'posix', '_frozen_importlib_external', '_thread', '_weakref', 'time', 'zipimport', '_codecs', 'codecs', 'encodings.aliases', 'encodings', 'encodings.utf_8', '_signal', '__main__', 'encodings.latin_1', '_abc', 'abc', 'io', '_stat', 'stat', '_collections_abc', 'genericpath', 'posixpath', 'os.path', 'os', '_sitebuiltins', '_locale', '_bootlocale', 'types', 'importlib._bootstrap', 'importlib._bootstrap_external', 'warnings', 'importlib', 'importlib.machinery', 'importlib.abc', '_operator', 'operator',

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.gradio.app:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): api.gradio.app:443
DEBUG:urllib3.connectionpool:https://api.gradio.app:443 "POST /gradio-initiated-analytics/ HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.gradio.app:443 "POST /gradio-launched-analytics/ HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://api.gradio.app:443 "POST /gradio-launched-telemetry/ HTTP/1.1" 200 None


Building prefix dict from the default dictionary ...


DEBUG:jieba:Building prefix dict from the default dictionary ...


Loading model from cache /tmp/jieba.cache


...

Loading model cost 1.218 seconds.


DEBUG:jieba:Loading model cost 1.218 seconds.


Prefix dict has been built successfully.


DEBUG:jieba:Prefix dict has been built successfully.
ʦ`⁼ə↓li↓↑ s`ɹ`↓ ʦʰə↓s`ɹ`↓ s`ɹ`↑ʧ⁼yeNN→.
 length:39
 length:39


/root/miniconda3/lib/python3.8/site-packages/gradio/processing_utils.py:175: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


# STEP 5 下载模型
## 本地部署方法请见[README](https://github.com/Plachtaa/VITS-fast-fine-tuning/blob/main/README_ZH.md)

In [2]:
%cd VITS-fast-fine-tuning/

/root/VITS-fast-fine-tuning


In [4]:
#@markdown ### 下载选项1：运行该单元格，浏览器会自动下载模型和配置文件
#@markdown ### Download option 1: Running this codeblock will download model & config files by your browser.
!python rearrange_speaker.py 
%run download_model.py

ModuleNotFoundError: No module named 'google.colab'

In [ ]:
#@markdown ### 下载选项2：运行该单元格会将模型和配置文件保存到Google云端硬盘
#@markdown ### Download option 2: Running this codeblock will save the mode & config files to your Google drive.
!python rearrange_speaker.py 
!cp ./G_latest.pth ../drive/MyDrive/G_latest.pth
!cp ./finetune_speaker.json ../drive/MyDrive/finetune_speaker.json
!cp ./moegoe_config.json ../drive/MyDrive/moegoe_config.json

In [3]:
%cd VITS-fast-fine-tuning

/root/VITS-fast-fine-tuning


In [4]:
#@markdown ### 运行该单元格会清空所有已上传的样本，需要时可使用
#@markdown ### Running this codeblock will delete all voice samples you have uploaded. Use it if you need.
!rm -rf ./custom_character_voice/*
!rm -rf ./video_data/*
!rm -rf ./raw_audio/*
!rm -rf ./denoised_audio/*
!rm -rf ./segmented_character_voice/*
!rm -rf long_character_anno.txt
!rm -rf short_character_anno.txt

In [ ]:
#@markdown ### 运行该单元格会将切片和标注复制到谷歌云端硬盘根目录下名为`voice_data`的文件夹下以用作其它用途
#@markdown ### Running this codeblock will copy all processed voices & annotations to a folder named `voice_data` under the root of Google Drive for other purpose of usage
!mkdir ../drive/MyDrive/voice_data/
!cp -rf ./custom_character_voice/ ../drive/MyDrive/voice_data/custom_character_voice/
!cp -rf ./segmented_character_voice/ ../drive/MyDrive/voice_data/segmented_character_voice/
!cp long_character_anno.txt ../drive/MyDrive/voice_data/long_character_anno.txt
!cp short_character_anno.txt ../drive/MyDrive/voice_data/short_character_anno.txt

In [ ]:
!pip install torch==1.13.1 torchvision==0.14.1 torchaudio==0.13.1